# <center> Transient Search with Difference Image Analysis </center>

###  <center> Bruno Sánchez, M. Domínguez, M. Lares </center>

####  <center> Friends of Friends Meeting </center>
####  <center> Córdoba 2017 </center>

## A transient is...
-------------------------------------

**Any astrophysical phenomena which happens only once and then decays**


<center><img src="plots/Supernova_Image_Light_Curve_Spectrum_Illustration_of.gif" width=600></center>

## A transient can be detected when...
-------------------------------------

**We have data for two different epochs, and we compare them directly**

$\Gamma \ ( \vec{D_{t_i}} \ , \vec{D_{t_{i+1}}} \ )$
 ---------------------------------------------------

*Our data are images taken from ground telescopes*


**But what kind of function is $\Gamma$??**


## *Difference Image Analysis* (DIA)

To compare we need a model for the images, and to choose the correct representation of data...

$$  Diff = \Gamma \ ( \ Ref, \ New \ )$$

* Alard & Lupton 1998: "A Method for Optimal Image Subtraction"

    $$ R \ (x, y) \ \otimes \ K \ (u, v) \cong N(x, y) $$
    
    $$ K \ (u, v) = \sum\limits_{n} a_n \ N(\mu=0, \sigma_u, \sigma_v) \ p_u(u) \ p_v(v) $$

$$ D = N - R \otimes \mathscr{F}^{-1} \left( \frac{\widehat{N}}{\widehat{K}} \right) $$ 

* Bramich 2008: "A new algorithm for difference image analysis"

    $$ N_{ij}=\sum\limits_{l, m} K_{lm}R_{i+l, j+m}  + B_0 $$